In [ ]:
import numpy as np
import random

def sigmoide(z):
    """La fonction d'activation sigmoide"""
    return 1.0/(1.0+np.exp(-z))

def sigmoide_derivee(z):
    """Derivative of the sigmoid function."""
    return sigmoide(z)*(1-sigmoide(z))

class RNA(object):

    def __init__(self, nc):
        """ nc[c] contient le nombre de neurones de la couche c, c = 0 ...nombre_couches-1
        la couche d'indice 0 est la couche d'entrée
        w[c] est la marice des poids entre la couche c et c+1
        w[c][i,j] est le poids entre le neuronne i de la couche c et j de la couche c+1
        i = 0 correspond au biais par convention
        les poids sont initialisés avec un nombre aléatoire selon une distribution N(0,1)
        """
        self.nombre_couches = len(nc)
        self.nc = nc
        self.w = [np.random.randn(x+1, y) for x, y in zip(nc[:-1], nc[1:])]

    def propagation_avant(self, a):
        """a est un vecteur d'activation. a[0]=1 correspond au biais"""
        for wc in self.w:
            a = np.vstack((np.ones(1),sigmoide(np.dot(wc.transpose(), a))))
        return a

    def SGD(self, donnees_entrainement, epochs, taille_mini_batch, eta,
            donnees_test=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``donnees_entrainement`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``donnees_test`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if donnees_test: n_test = len(donnees_test)
        n = len(donnees_entrainement)
        for j in range(epochs):
            random.shuffle(donnees_entrainement)
            mini_batches = [
                donnees_entrainement[k:k+taille_mini_batch]
                for k in range(0, n, taille_mini_batch)]
            for mini_batch in mini_batches:
                self.mini_batch(mini_batch, eta)
            if donnees_test:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(donnees_test), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def mini_batch(self, mini_batch, eta):
        """Update the network's w and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_w = [np.zeros(w.shape) for w in self.w]
        for x, y in mini_batch:
            delta_nabla_w = self.retropropagation(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.w = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.w, nabla_w)]

    def retropropagation(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.w``."""
        
        nabla_w = [np.zeros(wc.shape) for wc in self.w]
        # propagation avant avec stockage des activations[0] est à 1 pour les biais
        activation = np.vstack((np.ones(1),x)) # activation
        activations = [np.vstack((np.ones(1),x))] # liste des activations couche par couche
        zs = [] # # liste des z couche par couche

        for wc in w:
            z = np.dot(wc.transpose(), activation)
            zs.append(z)
            activation = np.vstack((np.ones(1),sigmoide(z))) 
            activations.append(activation)
        
        # rétropropagation 
        # delta est le vecteur des dérivées par rapport zjc-1
        # calcul de la dérivée par rapport a z pour la couche de sortie
        delta = derivee_cout(activations[-1][1:], y) * sigmoide_derivee(zs[-1])
        nabla_w[-1] = np.dot(activations[-2],delta)
        print("del w couche[2]:",nabla_w[-1])

        for l in range(2, nombre_couches):
            z = zs[-l]
            sp = sigmoide_derivee(z)
            delta = np.dot(w[-l+1], delta)[1:] * sp
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return nabla_w

    def evaluate(self, donnees_test):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in donnees_test]
        return sum(int(x == y) for (x, y) in test_results)

    def derivee_cout(output_activations, y):
    """ Vecteur des dérivées de la fonction de cout vs aj"""
    return (output_activations-y)
    
    def cost_derivative(self, output_activations, y):
    """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)


un_rna = RNA([2,3,1])